In [5]:
import os
import pdb
import time

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data.dataloader import DataLoader
from torch.utils.data.dataset import Dataset

from bc.beats import get_beat_bank
from bc.plot import plot_beat, plot_four_beats

device = torch.device("cuda:0")
torch.cuda.get_device_name(torch.cuda.current_device())

base_dir = os.path.abspath('..')
data_dir = os.path.join(base_dir, 'data')

# Table of record names and the beat types they contain
beat_table = pd.read_csv(os.path.join(data_dir, 'beat-types.csv'), dtype={'record':object})
beat_table.set_index('record', inplace=True)

In [6]:
# Load beats with fixed width.
n_beats, n_centers = get_beat_bank(data_dir=data_dir, beat_table=beat_table,
                                   wanted_type='N', filter=True, fixed_width=240)
l_beats, l_centers = get_beat_bank(data_dir=data_dir, beat_table=beat_table,
                                   wanted_type='L' ,filter=True, fixed_width=240)
r_beats, r_centers = get_beat_bank(data_dir=data_dir, beat_table=beat_table,
                                   wanted_type='R', filter=True, fixed_width=240)
v_beats, v_centers = get_beat_bank(data_dir=data_dir, beat_table=beat_table,
                                   wanted_type='V', filter=True, fixed_width=240)

In [ ]:
# Now all beats have the same number of samples
plot_four_beats(beats=[n_beats[0], l_beats[0], r_beats[0], v_beats[0]],
                centers=[n_centers[0], l_centers[0], r_centers[0], v_centers[0]],
                seconds=False)

In [ ]:
% matplotlib qt

In [ ]:
# Visualize how a beat will look as an M x 2 image
# Stretch a beat channelwise just for visualization
stretch_factor = 20
beat_image = np.empty([n_beats[0].shape[1] * stretch_factor, n_beats[0].shape[0]])
for i in range(stretch_factor):
    for ch in range(2):
        beat_image[ch*stretch_factor + i, :] = n_beats[0][:, ch]
plt.figure()
plt.imshow(beat_image, cmap='binary')
plt.show()

In [38]:
# Turn all beats into a large tensor
beats_all = torch.from_numpy(
    np.array(n_beats+ l_beats + r_beats + v_beats).transpose(0, 2, 1).astype('float32'))

In [39]:
labels_all = torch.tensor([0]*len(n_beats) + [1]*len(l_beats)
                            + [2]*len(r_beats) + [3]*len(v_beats), dtype=torch.int64)

In [9]:
# Define the neural network structure
class BeatNet(nn.Module):
    """
    Reference : https://pytorch.org/docs/master/nn.html#conv1d
    """
    def __init__(self):
        super(BeatNet, self).__init__()
        # input is 2 channel ecg
        self.conv1 = nn.Conv1d(in_channels=2, out_channels=4, kernel_size=3)
        self.conv2 = nn.Conv1d(in_channels=4, out_channels=8, kernel_size=3)
        # maxpool function
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2, padding=0)
        # First fully connected layer
        self.fc1 = nn.Linear(8 * 58, 12)
        # Final fully connected layer 
        self.fc2 = nn.Linear(12, 4)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        # Reshape input data for fully connected layer
        # Question: Why is this 8 * 58? Explain the steps.
        x = x.view(-1, 8 * 58)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    

In [10]:
# Define a Dataset class to iterate the beats
class BeatDataset(Dataset):
    def __init__(self, beats, labels):
        self.beats = beats
        self.labels = labels
        
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        'Generates one sample of data'
        x = self.beats[index]
        y = self.labels[index]
        return x, y

In [ ]:
# Visualize the data loader.
beatloader = DataLoader(dataset=BeatDataset(beats_all, labels_all),
                        shuffle=True, batch_size=4, num_workers=2)

for i, data in enumerate(beatloader, 0):        
    #Get inputs
    inputs, labels = data
    print(i, inputs.shape, labels)
    if i == 10:
        break

In [40]:
# Define the training function
def train_net(net, batch_size, n_epochs, learning_rate):
    #Print all of the hyperparameters of the training iteration:
    print("===== HYPERPARAMETERS =====")
    print("batch_size=", batch_size)
    print("epochs=", n_epochs)
    print("learning_rate=", learning_rate)
    print("=" * 30)
    
    # Create the loader instance to load our beat data
    beatloader = DataLoader(dataset=BeatDataset(beats_all, labels_all),
                            shuffle=True, batch_size=batch_size)
    # Create the loss function - cross entropy loss
    loss_fun = nn.CrossEntropyLoss()
    # Create the optimizer function - stochastic gradient descent
    optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)
    
    # The number of batches per epoch is the length of data divided by the batch size
    n_batches = len(beatloader)
    start_time = time.time()
    for epoch in range(n_epochs):
        running_loss = 0.0
        print_every = n_batches // 10
        start_time = time.time()
        total_train_loss = 0
        
        for i, data in enumerate(beatloader, 0):
            # Get inputs
            inputs, labels = data
            # Wrap them in a Variable object
            inputs, labels = Variable(inputs), Variable(labels)
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass, backward pass, optimize
            outputs = net(inputs)
            loss = loss_fun(outputs, labels)
            loss.backward()
            optimizer.step()

    print('Training finished in {:.2f}s'.format(time.time() - start_time))

In [18]:
labels_all.device

device(type='cpu')

In [41]:
beatnet = BeatNet()

beatnet.to(device)
beats_all = beats_all.to(device)
labels_all = labels_all.to(device)

In [42]:


train_net(beatnet, batch_size=4, n_epochs=2, learning_rate=0.001)

===== HYPERPARAMETERS =====
batch_size= 4
epochs= 2
learning_rate= 0.001
Training finished in 40.83s


In [43]:
testloader = DataLoader(dataset=BeatDataset(beats_all, labels_all),
                            shuffle=True, batch_size=4)

In [46]:
correct = 0
total = 0

for data, label in testloader:
#     data, label = beats_all[i], labels_all[i]
    outputs = beatnet(data)
    _, y_pred = torch.max(outputs, 1)
    total += label.size(0)
    correct += (y_pred == label).sum().item()

print('Accuracy of the network on the {} test items: {.2f}%'.format(
    len(testloader), 100 * correct / total))

ValueError: incomplete format

In [49]:
print('Accuracy of the network on the {} test items: {:.2f}%'.format(
    len(testloader), 100 * correct / total))

Accuracy of the network on the 21602 test items: 99.34%


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms


In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
trainset

In [ ]:
dataiter = iter(trainloader)
images, labels = dataiter.next()

In [ ]:

images.shape

In [ ]:
b = torch.from_numpy(np.array(v_beats))

In [ ]:

b.shape

In [ ]:
print(beatnet)

In [ ]:
# Image batch size is torch.Size([4, 3, 32, 32])
# So our ecg batch should be [4, 2, 240]

In [ ]:
v_beats[0].shape

In [ ]:
a = np.empty([4,2,3])

In [ ]:
a.transpose((0,2,1)).shape

In [ ]:
beats_all.shape

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
device

device(type='cpu')

0

'GeForce GTX 1060 6GB'